# Task 4: A* Beam Search for the 8-Puzzle Problem

This notebook implements and compares **A*** and **Beam-A*** for solving the 8-puzzle problem.

We evaluate:
- Path optimality
- Nodes expanded
- Runtime

The heuristic used is **Manhattan Distance**, which is admissible and consistent.

## 1. Imports and Global Definitions

In [ ]:
import heapq
import time
import random
import statistics
from typing import List, Tuple

GOAL_STATE = (1, 2, 3, 4, 5, 6, 7, 8, 0)
GOAL_POS = {GOAL_STATE[i]: (i // 3, i % 3) for i in range(9)}

## 2. Heuristic Function (Manhattan Distance)

In [ ]:
def manhattan(state: Tuple[int]) -> int:
    distance = 0
    for i, tile in enumerate(state):
        if tile != 0:
            x1, y1 = i // 3, i % 3
            x2, y2 = GOAL_POS[tile]
            distance += abs(x1 - x2) + abs(y1 - y2)
    return distance

## 3. Successor Function

In [ ]:
def get_neighbors(state: Tuple[int]) -> List[Tuple[int]]:
    neighbors = []
    idx = state.index(0)
    x, y = idx // 3, idx % 3
    moves = [(-1,0),(1,0),(0,-1),(0,1)]
    for dx, dy in moves:
        nx, ny = x + dx, y + dy
        if 0 <= nx < 3 and 0 <= ny < 3:
            nidx = nx * 3 + ny
            new_state = list(state)
            new_state[idx], new_state[nidx] = new_state[nidx], new_state[idx]
            neighbors.append(tuple(new_state))
    return neighbors

## 4. A* Algorithm Implementation

In [ ]:
def astar(start: Tuple[int]):
    open_list = []
    heapq.heappush(open_list, (manhattan(start), 0, start))
    g_cost = {start: 0}
    visited = set()
    nodes_expanded = 0

    start_time = time.time()

    while open_list:
        f, g, state = heapq.heappop(open_list)
        if state == GOAL_STATE:
            return g, nodes_expanded, time.time() - start_time
        if state in visited:
            continue
        visited.add(state)
        nodes_expanded += 1

        for neighbor in get_neighbors(state):
            ng = g + 1
            if neighbor not in g_cost or ng < g_cost[neighbor]:
                g_cost[neighbor] = ng
                heapq.heappush(open_list, (ng + manhattan(neighbor), ng, neighbor))

    return None

## 5. Beam-A* Algorithm Implementation

In [ ]:
def beam_astar(start: Tuple[int], beam_width: int):
    frontier = [(manhattan(start), 0, start)]
    g_cost = {start: 0}
    nodes_expanded = 0

    start_time = time.time()

    while frontier:
        frontier.sort(key=lambda x: x[0])
        frontier = frontier[:beam_width]
        next_frontier = []

        for f, g, state in frontier:
            if state == GOAL_STATE:
                return g, nodes_expanded, time.time() - start_time

            nodes_expanded += 1

            for neighbor in get_neighbors(state):
                ng = g + 1
                if neighbor not in g_cost or ng < g_cost[neighbor]:
                    g_cost[neighbor] = ng
                    next_frontier.append((ng + manhattan(neighbor), ng, neighbor))

        frontier = next_frontier

    return None

## 6. Random Puzzle Generator

In [ ]:
def generate_puzzle(moves=20):
    state = GOAL_STATE
    for _ in range(moves):
        state = random.choice(get_neighbors(state))
    return state

## 7. Experimental Evaluation

In [ ]:
def run_experiments(num_tests=10, beam_widths=[5, 10, 20]):
    results = {"A*": []}
    for bw in beam_widths:
        results[f"Beam-A* (k={bw})"] = []

    for _ in range(num_tests):
        start = generate_puzzle(25)

        res = astar(start)
        if res:
            results["A*"].append(res)

        for bw in beam_widths:
            res = beam_astar(start, bw)
            if res:
                results[f"Beam-A* (k={bw})"].append(res)

    return results

## 8. Summary Statistics

In [ ]:
def summarize(results):
    for algo, data in results.items():
        if not data:
            continue
        paths = [d[0] for d in data]
        nodes = [d[1] for d in data]
        times = [d[2] for d in data]

        print(f"\n{algo}")
        print(f"Path Length: {statistics.mean(paths):.2f} ± {statistics.stdev(paths) if len(paths)>1 else 0:.2f}")
        print(f"Nodes Expanded: {statistics.mean(nodes):.2f}")
        print(f"Time: {statistics.mean(times):.4f} s")

## 9. Run the Evaluation

In [ ]:
results = run_experiments(num_tests=10)
summarize(results)